# 모델 불러오기

각자 알아서 불러올 것

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import BertForSequenceClassification, ElectraForSequenceClassification, ElectraTokenizer
from tokenization_kobert import KoBertTokenizer

model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

# model = BertForSequenceClassification.from_pretrained("KoBERT_datav2/checkpoint-25000", **model_config)
# tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512)
model = ElectraForSequenceClassification.from_pretrained("KoELECTRA_base_v3_datav2/checkpoint-14000", **model_config)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", model_max_length=512)

In [2]:
# 테스트 데이터

test_df = pd.read_csv("data/test.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(50).tolist()

## 스피드 테스트

In [3]:
model.to("cpu")
model.eval()
print()

In [4]:
%%timeit

with torch.no_grad():
    tokens = tokenizer(speed_test_data, padding="max_length", truncation=True, return_tensors="pt")
    _ = model(**tokens)

23.1 s ± 252 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## F1 스코어

In [5]:
from data_loader import get_data_loaders


_, _, test_loader = get_data_loaders(tokenizer, use_imbalanced=False, device="cuda")

Using custom data configuration default-eafb513e725dbeca
Reusing dataset csv (C:\Users\cglab\.cache\huggingface\datasets\csv\default-eafb513e725dbeca\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at C:\Users\cglab\.cache\huggingface\datasets\csv\default-eafb513e725dbeca\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-d815bbee38dcbc26.arrow
Loading cached processed dataset at C:\Users\cglab\.cache\huggingface\datasets\csv\default-eafb513e725dbeca\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-5c9fdd80709ca508.arrow
Loading cached processed dataset at C:\Users\cglab\.cache\huggingface\datasets\csv\default-eafb513e725dbeca\0.0.0\9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff\cache-60328c4920383600.arrow


In [6]:
from torchmetrics import F1
from tqdm.auto import tqdm


f1_score = F1(num_classes=3, average="macro").cuda()
model.cuda()
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader):
        labels = batch.pop("labels")
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, dim=1)
        f1_score(pred, labels)

score = f1_score.compute()
print(score.item())

  0%|          | 0/157 [00:00<?, ?it/s]

0.9272494316101074
